In [1]:
import os

In [2]:
%pwd

'e:\\Text-Summarizer-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Text-Summarizer-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [11]:
!pip install -upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -u


Found existing installation: transformers 4.44.2
Uninstalling transformers-4.44.2:
  Successfully uninstalled transformers-4.44.2


  Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)
                                              0.0/324.4 kB ? eta -:--:--
     ---------                               81.9/324.4 kB 1.5 MB/s eta 0:00:01
     ----------------------------           245.8/324.4 kB 2.5 MB/s eta 0:00:01
     -------------------------------------- 324.4/324.4 kB 2.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

e:\Text-Summarizer-Project\textS\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-09-20 18:50:55,260]:INFO: config: PyTorch version 2.4.1 available.]


In [9]:
import os
import torch
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from datasets import load_from_disk

class ModelTrainer:
    def __init__(self, config):
        self.config = config
        self.model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt)
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model.to(self.device)
        
    def save_checkpoint(self, epoch):
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
        }
        torch.save(checkpoint, 'model_checkpoint.pth')
        print(f"Checkpoint saved at epoch {epoch}")

    def load_checkpoint(self):
        checkpoint = torch.load('model_checkpoint.pth')
        self.model.load_state_dict(checkpoint['model_state_dict'])
        return checkpoint['epoch']

    def train(self):
        # Load data
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_eval_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )

        trainer = Trainer(
            model=self.model,
            args=trainer_args,
            tokenizer=self.tokenizer,
            data_collator=DataCollatorForSeq2Seq(self.tokenizer, model=self.model),
            train_dataset=dataset_samsum_pt["train"],
            eval_dataset=dataset_samsum_pt["validation"]
        )

        start_epoch = 0
        # Check if there's a checkpoint to load
        if os.path.exists('model_checkpoint.pth'):
            start_epoch = self.load_checkpoint()
            print(f"Resuming training from epoch {start_epoch}")

        for epoch in range(start_epoch, self.config.num_train_epochs):
            try:
                trainer.train()
                # Save checkpoint after each epoch
                self.save_checkpoint(epoch + 1)  # Save checkpoint for next epoch
            except Exception as e:
                print(f"An error occurred: {e}")
                # Save state here if desired
                self.save_checkpoint(epoch + 1)
                break  # Exit the loop to pause

        # Save final model and tokenizer
        self.model.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        self.tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))


In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()

except Exception as e:
    raise e

[2024-09-20 18:50:59,178]:INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-20 18:50:59,186]:INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-20 18:50:59,191]:INFO: common: created directory at: artifacts]
[2024-09-20 18:50:59,191]:INFO: common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
e:\Text-Summarizer-Project\textS\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


AttributeError: 'ModelTrainerConfig' object has no attribute 'per_device_eval_batch_size'